# Olympische Athleten-Datenanalyse

## Datenvorbereitungsphase

### Schritt 1: Daten aus der ursprünglichen CSV-Datei laden

In [1]:
import pandas as pd

data = pd.read_csv('olympics_cleaned_v4.csv')

data.head()

,ID,Name,Sex,Age,Height,Weight,NOC,Year,Sport,Event,Medal,country
0,1,A Dijiang,M,24.0,180.0,80.0,CHN,1992,Basketball,Basketball Men's Basketball,NaN,China
1,2,A Lamusi,M,23.0,170.0,60.0,CHN,2012,Judo,Judo Men's Extra-Lightweight,NaN,China
2,602,Abudoureheman,M,22.0,182.0,75.0,CHN,2000,Boxing,Boxing Men's Middleweight,NaN,China
3,1463,Ai Linuer,M,25.0,160.0,62.0,CHN,2004,Wrestling,"Wrestling Men's Lightweight, Greco-Roman",NaN,China
4,1464,Ai Yanhan,F,14.0,168.0,54.0,CHN,2016,Swimming,Swimming Women's 200 metres Freestyle,NaN,China


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219085 entries, 0 to 219084
Data columns (total 12 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   ID       219085 non-null  int64  
 1   Name     219085 non-null  object 
 2   Sex      219085 non-null  object 
 3   Age      210810 non-null  float64
 4   Height   170149 non-null  float64
 5   Weight   168197 non-null  float64
 6   NOC      219085 non-null  object 
 7   Year     219085 non-null  int64  
 8   Sport    219085 non-null  object 
 9   Event    219085 non-null  object 
 10  Medal    33610 non-null   object 
 11  country  219064 non-null  object 
dtypes: float64(3), int64(2), object(7)
memory usage: 20.1+ MB


Nachdem wir die 'info'-Methode verwendet haben, sehen wir, dass in einigen Spalten Werte fehlen.

### Schritt 2: Daten in Tabellen aufteilen

Ursprünglich wir haben eine große Tabelle. Das Arbeiten mit einer so großen Tabelle ist ineffizient und speicherintensiv. 
Um Daten effizient und korrekt zu speichern, soll man Daten auf die kleine normalizirunge Tabellen aufteilen. Ich habe beschlossen, die folgenden Tabellen zu erstellen:

- athletes.csv
- athlets_events.csv
- countries.csv
- events.csv

Solche Aufeilung darstellt logische Struktur der Datenbank. Die Daten sinnvoll aufgeteilt werden und jede Entität nur die relevanten Informationen enthält.

Zuerst bereinigen wir die Daten und konvertieren sie in den gewünschten Typ

In [3]:
data = data.rename(columns={'ID': 'ID_athlet'})# Benennen Sie die Spalte „ID“ in „ID_athlet“ um

#Ersetzt alle leeren Zeichenfolgen und Zeichenfolgen, die Leerzeichen enthalten, durch NaN.
data["Height"] = data['Height'].replace(['', ' '], pd.NA) 
data["Weight"] = data["Weight"].replace(['', ' '], pd.NA)
data['Age'] = data['Age'].replace(['', ' '], pd.NA)
data['country'] = data['country'].replace(['', ' '], pd.NA)

#Bearbeiten wir zunächst die Datentypen, sodass sie besser geeignet sind
data['ID_athlet'] = data['ID_athlet'].astype('int32')  
data['Year'] = data['Year'].astype('int32')  
data['Sex'] = data['Sex'].astype('object')  
data['NOC'] = data['NOC'].astype('object')  
data['Sport'] = data['Sport'].astype('object')  
data['Event'] = data['Event'].astype('object')  
data['Medal'] = data['Medal'].astype('object')  
data['country'] = data['country'].astype('object')  
data['Age'] = data['Age'].astype('float32')
data['Height'] = pd.to_numeric(data['Height'], errors='coerce')#Konvert Werte in ein numerisches Format
data['Weight'] = pd.to_numeric(data['Weight'], errors='coerce')
data['Age'] = pd.to_numeric(data['Age'], errors='coerce')

Erstellen einen Datenrahmen für jede einzelne Entität

In [4]:
# 1 Table: athlets
athletes = data[['ID_athlet', 'Name', 'Sex', 'Height', 'Weight']].drop_duplicates() 

#2 Table: countries
countries = data[['NOC', 'country']].drop_duplicates() 
# 3 Table: events 
events = data[['Sport', 'Event']].drop_duplicates().reset_index() 
events.rename(columns={'index': 'ID_event'}, inplace=True) 

# assign index
events['ID_event'] = events['ID_event'].astype('int32') 

# 4 Relation table: athletes_events
athletes_events = data[['ID_athlet', 'Event', 'Year', 'Medal', 'Age', 'NOC']].drop_duplicates() 

Wenn wir 'data' geteilt haben, haben wir in der 'athletes_events' eine 'event' Spalte, die wir in 'ID_event' ändern müssen.

In [5]:
# athletes_events und events verbinden, um ID_event zu erhalten
athletes_events_with_ids = athletes_events.merge(events, on='Event', how='left') 

# Lassen wir nur benötigte Spalten
athletes_events_final = athletes_events_with_ids[['ID_athlet', 'ID_event', 'Year', 'Medal', 'Age', 'NOC']]


### Schritt 3: Tabellen in separaten CSV-Dateien speichern

In [6]:
athletes.to_csv('athletes.csv', sep=';',index=False)
countries.to_csv('countries.csv', sep=';', index=False)
events.to_csv('events.csv', sep=';', index=False)
athletes_events_final.to_csv('athletes_events.csv', sep=';', index=False)